<a href="https://colab.research.google.com/github/stuartgreen4j/bug-free-potato/blob/main/norfolk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Norfolk Data Model

## Initialise

In [ ]:
!pip install neo4j
!pip install faker

NEO4J_URI='neo4j+s://83b82ac7.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='9dUIw21Z9LRgmxGo3JUDiwtifZyJCEk8jedQedLEaxs'

from neo4j import GraphDatabase
import pandas as pd
import random
import datetime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Start neo4j session

In [ ]:
#n4j = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)).session(database='hmrc')
n4j = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)).session()

## Create all required indexes

In [ ]:
indexes = [
    'CREATE INDEX IF NOT EXISTS FOR (n:PostCode) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:Outcode) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:GivenName) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:GivenNameSoundex) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:GivenNameSource) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:GivenNameSoundexSource) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:FamilyName) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:FamilyNameSoundex) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:FamilyNameSource) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:FamilyNameSoundexSource) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:Day) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:BirthDaySource) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:Month) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:Year) ON (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:Address) on (n.addressLine)',
    'CREATE INDEX IF NOT EXISTS FOR (n:AddressSource) on (n.value)',
    'CREATE INDEX IF NOT EXISTS FOR (n:IdentifierSource) on (n.value)',
]

list(map(n4j.run, indexes))

[]

## Load Postcode data

In [ ]:
cypher = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/stuartgreen4j/bug-free-potato/main/nrpc.csv" as r
CREATE (p:PostCode {population: toInteger(r.Population), value: r.Postcode, households: toInteger(r.Households) })
"""
n4j.run(cypher)

## Create Outcodes

In [ ]:
cypher="""
MATCH (n:PostCode)
WITH DISTINCT head(apoc.text.split(n.value, ' ')) as outcode
CREATE (o:Outcode {value: outcode})
"""
n4j.run(cypher)

## Link Outcode to Postcode

In [ ]:
cypher = """
MATCH (n:PostCode)
WITH head(apoc.text.split(n.value, ' ')) as outcode, n
MATCH (o:Outcode {value: outcode})
CREATE (o)<-[:WITHIN]-(n)
"""
n4j.run(cypher)

## Load postcode, population & household data from graph

In [ ]:
cypher = """
MATCH (p:PostCode) WHERE p.households > 0
RETURN p.value as code, p.population as population, p.households as households
"""
result = n4j.run(cypher)
postcodes = pd.DataFrame([record.data() for record in result])

## Create fake data

In [ ]:
# Initialise
#npeople = 1000000
npeople = 1
seed = 123456

from faker import Faker
fake = Faker(['en_GB'])
Faker.seed(seed)
random.seed(seed)

# Weighted random distribution of postcode based on population
pcl = random.choices(postcodes[['code','households']].values, weights=postcodes['population'], k=npeople)

# Ramdomise address based on number of households
addressl = list(map(lambda x: f'{random.randrange(x[1])} {x[0]}', pcl))

# Reduce postcode list to just postcode
pcl = list(map(lambda x: x[0], pcl))

# Create fake names
namel = list(map(lambda x: f'{fake.first_name()} {fake.last_name()}', range(npeople)))

# Create fake national insurance number
natinsl = list(map(lambda x: fake.ssn().replace(" ", ""), range(npeople)))

# Create fake DoB
start = datetime.date(1906, 1, 1)
end = datetime.date(2006, 1, 1)
bdl = list(map(lambda x: fake.date_between(start, end), range(npeople)))

# Create csv
df = pd.DataFrame(zip(pcl, namel, bdl, natinsl, addressl), columns=['postcode', 'name', 'dob', 'natins', 'address'])
df.to_csv("nrpeople.csv")

## Load people to graph

In [ ]:
# Done in 4 chunks due to size limitations of github
csvs = [
    "https://raw.githubusercontent.com/stuartgreen4j/bug-free-potato/main/nr250k1.csv",
    "https://raw.githubusercontent.com/stuartgreen4j/bug-free-potato/main/nr250k2.csv",
    "https://raw.githubusercontent.com/stuartgreen4j/bug-free-potato/main/nr250k3.csv",
    "https://raw.githubusercontent.com/stuartgreen4j/bug-free-potato/main/nr250k4.csv",
    ]

cypher = """
LOAD CSV WITH HEADERS FROM $csvfile as r
MATCH (c:PostCode {value: r.postcode})
CREATE (p:Person {name: r.name, dob: r.dob})
CREATE (i:Identifier {value: r.natins})
MERGE (is:IdentifierSource {value: 'neo4j PS'})
CREATE (p)-[:HAS_IDENTIFIER]->(i)
MERGE (p)-[:HAS_IDENTIFIER_SOURCE]->(is)
MERGE (i)-[:HAS_IDENTIFIER_SOURCE]->(is)
MERGE (a:Address {addressLine: r.address})
MERGE (as:AddressSource {value: 'neo4j PS'})
CREATE (p)-[:HAS_ADDRESS]->(a)
MERGE (p)-[:HAS_ADDRESS_SOURCE]->(as)
MERGE (a)-[:HAS_ADDRESS_SOURCE]->(as)
MERGE (a)-[:HAS_POSTCODE]->(c)
"""
list(map(lambda x: n4j.run(cypher, csvfile=x), csvs))


## Create FamilyName and associated Soundex

In [ ]:
cypher = """
MATCH (p:Person) 
WITH apoc.text.split(p.name, " ") AS name, p
MERGE (fn:FamilyName {value: name[1]})
MERGE (fns:FamilyNameSource {value: 'neo4j PS'})
MERGE (fnx:FamilyNameSoundex {value: apoc.text.phonetic(name[1])})
MERGE (fnxs:FamilyNameSoundexSource {value: 'neo4j Soundex'})
MERGE (p)-[:HAS_FAMILY_NAME]->(fn)
MERGE (p)-[:HAS_FAMILY_NAME_SOURCE]->(fns)
MERGE (fn)-[:HAS_FAMILY_NAME_SOURCE]-(fns)
MERGE (p)-[:HAS_FAMILY_NAME_SOUNDEX]->(fnx)
MERGE (p)-[:HAS_FAMILY_NAME_SOUNDEX_SOURCE]->(fnxs)
MERGE (fnx)-[:HAS_FAMILY_NAME_SOUNDEX_SOURCE]-(fnxs)
MERGE (fn)-[:HAS_SOUNDEX]->(fnx)
"""
n4j.run(cypher)

## Create GivenName and associated Soundex

In [ ]:
cypher = """
MATCH (p:Person) 
WITH apoc.text.split(p.name, " ") AS name, p
MERGE (gn:GivenName {value: name[0]})
MERGE (gns:GivenNameSource {value: 'neo4j PS'})
MERGE (gnx:GivenNameSoundex {value: apoc.text.phonetic(name[1])})
MERGE (gnxs:GivenNameSoundexSource {value: 'neo4j Soundex'})
MERGE (p)-[:HAS_GIVEN_NAME]->(gn)
MERGE (p)-[:HAS_GIVEN_NAME_SOURCE]->(gns)
MERGE (gn)-[:HAS_GIVEN_NAME_SOURCE]-(gns)
MERGE (p)-[:HAS_GIVEN_NAME_SOUNDEX]->(gnx)
MERGE (p)-[:HAS_GIVEN_NAME_SOUNDEX_SOURCE]->(gnxs)
MERGE (gnx)-[:HAS_GIVEN_NAME_SOUNDEX_SOURCE]-(gnxs)
MERGE (gn)-[:HAS_SOUNDEX]->(gnx)
"""
n4j.run(cypher)

## Create DoB nodes and relationships

In [ ]:
cypher = """
MATCH (p:Person)
WITH apoc.text.split(p.dob,"-") as dob, p
MERGE (y:Year {value: toInteger(dob[0])})
MERGE (m:Month {value: toInteger(dob[1])})
MERGE (d:Day {value: toInteger(dob[2])})
MERGE (ds:BirthDaySource {value: 'neo4j PS'})
MERGE (p)-[:HAS_BIRTH_YEAR]->(y)
MERGE (p)-[:HAS_BIRTH_MONTH]->(m)
MERGE (p)-[:HAS_BIRTH_DAY]->(d)
MERGE (p)-[:HAS_BIRTH_DAY_SOURCE]->(ds)
MERGE (d)-[:HAS_BIRTH_DAY_SOURCE]->(ds)
MERGE (d)-[:WITHIN_MONTH]->(m)
MERGE (m)-[:WITHIN_YEAR]->(y)
"""
n4j.run(cypher)

KeyboardInterrupt: ignored

## Close neo4j session

In [ ]:
n4j.close()